# 언어 모델기반 임베딩

- 언어 모델은 단어 시퀀스에 대한 확률 분포
- 주어진 특정 단어 시퀀스에 대해 다음 단어를 예측해볼 수 있다.(언어 모델이 양방향인 경우 이전 단어도)
- 대규모 자연 문법 구조를 활용하기 때문에 능동 레이블을 포함하지 않으므로 어떤의미에서는 비지도학습
- 단어 임베딩으로서의 언어 모델과 좀 더 전통적인 임베딩의 주된 차이점은 전통적인 임베딩은 단일 초기 변환으로 데이터에 적용된 다음 특정 작업에 특화돼 미세조정된다. 반면 언어 모델은 대규모 외부 말뭉치로 훈련받으며 특정언어 모델을 나타냄. 이를 사전훈련이라고 한다.
- 하지만 이러한 언어 모델을 사전 훈련하는 연산비용은 대개 상당히 높다. 그리하여 공개된 사전훈련모델을 미세조정 하면된다.
1. 자신의 분야에 맞는 텍스트를 사용해 마지막 몇 개의 계층을 재교육
2. 언어 모델의 마지막 계층을 제거하고 이를 하나 또는 두개의 완전 연결망으로 대체. 이 연결망은 과제에 따라 입력의 언어모델 임베딩을 최종 범주형 또는 회귀로 변환하는 일을 한다.

In [5]:
#!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.20 KiB | 525.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


## BERT를 자신의 신경망 일부로 사용

In [6]:
#!pip install transformers

     |████████████████████████████████| 674 kB 665 kB/s 
     |████████████████████████████████| 690 kB 2.8 MB/s 
     |████████████████████████████████| 883 kB 2.2 MB/s 
     |████████████████████████████████| 1.2 MB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 3.1 MB/s 
  Created wheel for regex: filename=regex-2020.6.8-cp37-cp37m-macosx_10_9_x86_64.whl size=283189 sha256=9897d850c7ac0f1b4cc05519a221db6c65c70fa87de822ad7dcfa8e118b290ac
  Stored in directory: /Users/HumanRevolution/Library/Caches/pip/wheels/46/f1/0b/a372e98f7103934a3573301c71b475143baf8ba6f6dffc876c
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=11b0fd52d825b8def4ec37c0df388f0657130f3d0b2cec2aae64173ce4362ecc
  Stored in directory: /Users/HumanRevolution/Library/Caches/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built regex sacremoses


In [ ]:
# https://pytorch.org 에서 Quick Start Locally라는 제목 부분을 찾아 코드를 cmd에 입력

In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, BertForSequenceClassification, glue_convert_examples_to_features

In [2]:
# 코드 후밤ㄴ부에 사용하게 될 몇가지 상수를 선언
BATCH_SIZE = 32
FINE_TUNED_MODEL_DIR = './bert/model'

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")

In [4]:
# 텐서플로 Datasets를 통해 데이터셋 로드
data, info = tfds.load('glue/mrpc', with_info=True)
num_train = info.splits['train'].num_examples
num_valid = info.splits['validation'].num_examples

INFO:absl:Load dataset info from /Users/HumanRevolution/tensorflow_datasets/glue/mrpc/1.0.0
INFO:absl:Reusing dataset glue (/Users/HumanRevolution/tensorflow_datasets/glue/mrpc/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /Users/HumanRevolution/tensorflow_datasets/glue/mrpc/1.0.0


In [5]:
# GLUE용 데이터셋을 tf.data.Dataset 인스턴스로 준비
Xtrain = glue_convert_examples_to_features(data['train'], tokenizer, 128, 'mrpc')
Xtrain = Xtrain.shuffle(128).batch(BATCH_SIZE).repeat(-1)
Xvalid = glue_convert_examples_to_features(data['validation'], tokenizer, 128, 'mrpc')
Xvalid = Xvalid.batch(BATCH_SIZE)

In [6]:
Xtrain

<RepeatDataset shapes: ({input_ids: (None, None), attention_mask: (None, None), token_type_ids: (None, None)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

In [7]:
Xvalid

<BatchDataset shapes: ({input_ids: (None, None), attention_mask: (None, None), token_type_ids: (None, None)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

- 다음으로 손실함수, 최적화기와 행렬을 정의하고 몇 훈련 에폭동안 모델을 적합화함.
- 모델을 미세 조정하고 있으므로 에폭의 수는 2개에 불과하며 학습률도 매우 작다.

In [17]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])
train_steps = num_train // BATCH_SIZE
valid_steps = num_valid // BATCH_SIZE
history = model.fit(Xtrain, epochs=2, steps_per_epoch=train_steps,
    validation_data=Xvalid, validation_steps=valid_steps)

Epoch 1/2
114/114 [==============================] - 1465s 13s/step - loss: 0.5846 - accuracy: 0.6796 - val_loss: 0.4892 - val_accuracy: 0.7865
Epoch 2/2
114/114 [==============================] - 1315s 12s/step - loss: 0.3855 - accuracy: 0.8193 - val_loss: 0.3712 - val_accuracy: 0.8438


In [22]:
# 미세조정된 모델을 'bert/data'에 저장
model.save_pretrained(FINE_TUNED_MODEL_DIR)

- 한 쌍의 문장이 서로의 문단임을 예측하고자 모델을 파이토치 모델로 다시 로드한다.
- from_tf = True 매개변수는 저장된 모델이 텐서플로 체크포인트임을 나타낸다.
- 다음은 문장의 쌍이 각각의 문단인 (sentence_0, sentence_1)과 그렇지 않은 쌍인 (sentence_0, sentence_2)를 사용해 저장된 모델을 테스트한다.

In [8]:
# 모세조정되어 저장된 모델 로드
saved_model = BertForSequenceClassification.from_pretrained(FINE_TUNED_MODEL_DIR, from_tf=True)

In [11]:
def print_result(id1, id2, pred):
    if pred == 1:
        print("Sentence_{:d} is a paraphrase of sentence_{:d}".format(id2, id1))
    else:
        print("Sentence_{:d} is not a paraphrase of sentence_{:d}".format(id2, id1))

sentence_0 = "At least 12 people were killed in the battle last week."
sentence_1 = "At least 12 people lost their lives in last weeks fighting."
sentence_2 = "The fires burnt down the houses on the street."
sentence_3 = "A bomb is planted in my ear"

inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=False, return_tensors="pt")
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=False, return_tensors="pt")
inputs_3 = tokenizer.encode_plus(sentence_0, sentence_3, add_special_tokens=False, return_tensors="pt")

pred_1 = saved_model(**inputs_1)[0].argmax().item()
pred_2 = saved_model(**inputs_2)[0].argmax().item()
pred_3 = saved_model(**inputs_3)[0].argmax().item()

print_result(0, 1, pred_1)
print_result(0, 2, pred_2)
print_result(0, 3, pred_3)

Sentence_1 is a paraphrase of sentence_0
Sentence_2 is not a paraphrase of sentence_0
Sentence_3 is not a paraphrase of sentence_0
